<a href="https://colab.research.google.com/github/ggieun/h.e.after/blob/main/CLV%EC%8B%9C%EB%8F%84_%ED%96%88%EC%9C%BC%EB%82%98_%EC%8B%A4%ED%8C%A8%ED%95%9C_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lifetimes
!pip install sqlalchemy

In [ ]:
from sqlalchemy import create_engine
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
from lifetimes.utils import summary_data_from_transaction_data

In [ ]:
data_inventory = pd.read_parquet('/content/drive/MyDrive/inventory_items_optimized.parquet')
data_items = pd.read_parquet('/content/drive/MyDrive/order_items_optimized.parquet')

In [ ]:
data_inventory.head(2)

In [ ]:
data_items.head(2)

In [ ]:
# 두데이터 Product_id로 조인
# 같은 product_id 마다 조인이 되서 값이 ROW가 300만개 생김

merged_data = pd.merge(data_items, data_inventory, on='product_id')
merged_data

In [ ]:
# order_id 중복값 제거 및 데이트 형식 변경
merged_data = merged_data.drop_duplicates(subset=['order_id'])
merged_data['invoiceDate'] = pd.to_datetime(merged_data['created_at_x'])
#merged_data

In [289]:
#profit 계산
merged_data['profit'] = merged_data['product_retail_price']-merged_data['cost']

In [290]:
# status 에서 cancelled 만 뺀 데이터

cancellations = merged_data[merged_data['status'] == 'Cancelled']

In [291]:
# status 에서 returned 만 뺀 데이터

returned = merged_data.dropna(subset=['returned_at'])
returned['returned_at']

273              2023-10-17 00:08:00+00:00
333              2024-01-18 13:53:00+00:00
398              2023-05-20 07:23:00+00:00
937              2022-01-28 15:01:00+00:00
1042             2023-03-13 02:11:00+00:00
                        ...               
3588013          2023-11-07 20:40:00+00:00
3588143          2023-02-27 06:59:00+00:00
3588200          2022-03-14 03:38:00+00:00
3588331   2024-01-19 17:31:49.530978+00:00
3588683          2020-12-30 22:43:00+00:00
Name: returned_at, Length: 12530, dtype: datetime64[ns, UTC]

In [292]:
#전체 프레임에서 cancellations 뺀 데이터
valid_orders = merged_data[~merged_data['order_id'].isin(cancellations['order_id'])]

In [293]:
##전체 프레임에서 returned 뺀 데이터 : 최종 활용 데이터
valid_orders_fin = valid_orders[~valid_orders['order_id'].isin(returned['order_id'])]

In [294]:
valid_orders_fin.sort_values('user_id')

,id_x,order_id,user_id,product_id,inventory_item_id,status,created_at_x,shipped_at,delivered_at,returned_at,...,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id,invoiceDate,profit
429105,1,1,1,2953,3,Shipped,2022-07-20 10:32:05+00:00,2022-07-20 16:42:00+00:00,NaT,NaT,...,6.000000,Active,Tommy Hilfiger Women's 3 Pack Sports Crew,Tommy Hilfiger,15.000000,Women,E2F9247929B404B2FE98BA6F32301E3B,1,2022-07-20 10:32:05+00:00,9.000000
772612,4,2,2,25774,11,Complete,2022-02-20 10:28:57+00:00,2022-02-22 12:23:00+00:00,2022-02-23 18:29:00+00:00,NaT,...,10.384000,Underwear,Tommy Bahama Tropical Print Boxer Short,Tommy Bahama,22.000000,Men,5F6FC587BA2A036B38AFFBB8C0A42008,4,2022-02-20 10:28:57+00:00,11.616000
3124372,6,4,3,26696,15,Processing,2023-08-08 06:13:20+00:00,NaT,NaT,NaT,...,47.306250,Sleep & Lounge,Tommy Bahama Terry Loop Robe,Tommy Bahama,108.750000,Men,A03175E68893309A3C69D0B9E018BCCA,4,2023-08-08 06:13:20+00:00,61.443750
3139231,7,5,3,18177,18,Shipped,2023-03-10 07:14:45+00:00,2023-03-11 12:33:00+00:00,NaT,NaT,...,49.825469,Active,Smartwool Men's Midweight Bottom,SmartWool,109.989998,Men,8288EB462E80418E70A393D245A984CE,2,2023-03-10 07:14:45+00:00,60.164529
2483466,5,3,3,22308,13,Processing,2023-04-23 09:04:57+00:00,NaT,NaT,NaT,...,30.174971,Pants,Dockers Men's Limited Offer D2 Stretch Khaki Pant,Dockers,59.990002,Men,4B6E16D36F691EEC61154D01871CEC76,7,2023-04-23 09:04:57+00:00,29.815031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1825141,181753,125221,99998,19164,490690,Processing,2022-12-16 06:35:21+00:00,NaT,NaT,NaT,...,20.760000,Sweaters,Original Penguin Men's Kris Sweater,Original Penguin,40.000000,Men,8F9FE000818954F238235F26CA070247,2,2022-12-16 06:35:21+00:00,19.240000
2351303,181754,125222,99998,16938,490692,Processing,2023-01-28 06:25:04+00:00,NaT,NaT,NaT,...,28.930000,Tops & Tees,Woolrich Men's Elite Discreet Carry Short Slee...,Woolrich,55.000000,Men,7E53526FC58A32E5452294826783A54B,8,2023-01-28 06:25:04+00:00,26.070000
767815,181755,125223,99998,21923,490694,Processing,2023-07-22 05:16:01+00:00,NaT,NaT,NaT,...,10.027440,Pants,Allegra K Men Belt Loop Pleats Slant Pockets C...,Allegra K,21.990000,Men,6D7C8FCB2F7A9E4126A6BE9777C5CA49,9,2023-07-22 05:16:01+00:00,11.962560
1001991,181756,125224,99999,25959,490697,Shipped,2023-12-23 08:59:09+00:00,2023-12-24 00:44:00+00:00,NaT,NaT,...,11.875000,Underwear,Hanes Men's 3 Pack Comfortblend Long Leg Boxer...,Hanes,25.000000,Men,F2E939D15E4F4D852C246293137196BE,3,2023-12-23 08:59:09+00:00,13.125000


In [295]:
valid_orders_fin['Date'] = pd.to_datetime(valid_orders_fin['invoiceDate']).dt.strftime('%Y-%m-%d')
valid_orders_fin['Date'] = pd.to_datetime(valid_orders_fin['Date'])


In [296]:
# T값 구하기
from datetime import datetime
current_date = datetime.now()
import pytz
valid_orders_fin['T'] = valid_orders_fin.apply(lambda row: (current_date - row['Date']).days, axis=1)
valid_orders_fin['T']

16          386
32          120
48          103
64          654
80          197
           ... 
3588484    1231
3588584     291
3588620     293
3588725     143
3588746     752
Name: T, Length: 94087, dtype: int64

In [297]:
# user_id  groupby (주문 횟수, 총 수익)
valid_orders_fin = valid_orders_fin.groupby(['user_id']).agg( {'order_id' : 'count', 'profit' : 'sum' , 'invoiceDate':'max' , 'T' :'max'}).reset_index()
valid_orders_fin.sort_values(by='order_id', ascending=False)

,user_id,order_id,profit,invoiceDate,T
23522,35527,4,56.378890,2022-12-22 00:16:50+00:00,1287
13856,20827,4,168.206310,2023-04-04 01:37:30+00:00,1321
26957,40732,4,142.536478,2023-10-30 08:29:14+00:00,329
64265,96985,4,100.955080,2022-06-30 06:55:14+00:00,1134
44454,67024,4,107.949562,2023-04-05 07:22:03+00:00,707
...,...,...,...,...,...
26009,39299,1,9.486000,2022-12-09 07:45:45+00:00,481
26010,39300,1,2.895750,2023-10-02 13:02:31+00:00,184
26011,39303,1,33.410000,2023-01-23 17:23:04+00:00,436
26012,39304,1,11.000000,2023-11-27 02:13:21+00:00,128


In [323]:
from lifetimes.utils import summary_data_from_transaction_data
from lifetimes import GammaGammaFitter

# 고객별 RFM 데이터 생성

valid_orders_fin['invoiceDate'] = pd.to_datetime(valid_orders_fin['invoiceDate']).dt.strftime('%Y-%m-%d')
valid_orders_fin_cleaned = valid_orders_fin[valid_orders_fin['profit'] > 0]
summary = summary_data_from_transaction_data(valid_orders_fin_cleaned, 'user_id', 'invoiceDate', monetary_value_col='profit')
summary['monetary_value'] = summary['monetary_value'].clip(lower=0.01)


#T = valid_orders_fin['T']
#summary
# RFM 모델 학습
bgf = BetaGeoFitter(penalizer_coef=0.1)
bgf.fit(summary['frequency'], summary['recency'], summary['T'])

##from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#scaled_data = scaler.fit_transform(summary[['frequency', 'recency', 'T']])


#from lifetimes import ParetoNBDFitter
# Pareto/NBD 모델 생성
#pareto_nbd = ParetoNBDFitter()

# 모델 학습
#pareto_nbd.fit(summary['frequency'], summary['recency'], summary['T'])

# Gamma-Gamma 모델로 평균 주문 가치 예측

ggf = GammaGammaFitter(penalizer_coef=1000000000000000000)
ggf.fit(summary['frequency'], summary['monetary_value'], iterative_fitting=1000000000000000000)

# 평균 주문 가치를 기반으로 각 고객의 평균 수익 계산
summary['expected_avg_profit'] = ggf.conditional_expected_average_profit(summary['frequency'], summary['monetary_value'])

# 예상 평균 수익을 기반으로 CLV 계산 (예시: 12개월)
summary['predicted_clv'] = summary['expected_avg_profit'] * 12

# 결과 출력
print(summary)


  message: NaN result encountered.
  success: False
   status: 3
      fun: nan
        x: [ 1.000e-01  1.000e-01  1.000e-01]
      nit: 0
      jac: [       nan  2.443e+18  2.443e+18]
 hess_inv: [[1 0 0]
            [0 1 0]
            [0 0 1]]
     nfev: 1
     njev: 1


/usr/local/lib/python3.10/dist-packages/lifetimes/fitters/__init__.py:101: OptimizeWarning: Unknown solver options: iterative_fitting
  output = minimize(
/usr/local/lib/python3.10/dist-packages/autograd/tracer.py:48: RuntimeWarning: divide by zero encountered in log
  return f_raw(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/autograd/tracer.py:48: RuntimeWarning: invalid value encountered in multiply
  return f_raw(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/autograd/numpy/numpy_vjps.py:34: RuntimeWarning: invalid value encountered in multiply
  defvjp(anp.multiply,    lambda ans, x, y : unbroadcast_f(x, lambda g: y * g),


ConvergenceError: 
The model did not converge. Try adding a larger penalizer to see if that helps convergence.
